# ServiceTitan Data Science Internship 
## Alissa Jouljian 

In [1]:
import pandas as pd
import pickle

#### invoice_id: int
#### created_on: datetime64[ns]
#### invoiceitem_id: int
#### invoiceitem_name: str
#### type: str (use this conversion table: {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})
#### unit_price: int
#### total_price: int (unit_price*quantity)
#### percentage_in_invoice: float (unit_price*quantity / invoice_total)
#### is_expired: bool (whether invoice_id is contained expired_invoices.txt or not)

In [2]:
with open('invoices_new.pkl' , 'rb') as i:
    invoices = pickle.load(i)
invoices

[{'id': '365371O',
  'created_on': '2019-05-27',
  'items': [{'item': {'id': 142446,
     'name': 'ii_142446',
     'unit_price': 182,
     'type': 1},
    'quantity': 6},
   {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
    'quantity': 1}]},
 {'id': 389528,
  'created_on': '2019-03-19',
  'items': [{'item': {'id': 172535,
     'name': 'ii_172535',
     'unit_price': 105,
     'type': 0},
    'quantity': 4},
   {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0},
    'quantity': 7},
   {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2},
    'quantity': 4},
   {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2},
    'quantity': 6},
   {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1},
    'quantity': 7},
   {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1},
    'quantity': 8},
   {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 1

In [3]:
with open('expired_invoices.txt' , 'r') as e:
    expired =  e.read().splitlines()

In [4]:
int_list = [int(value) for value in expired[0].split(', ')]

In [5]:

class DataExtractor():
    def __init__(self, invoice, expired):
        self.invoice_name = invoice 
        self.expired_name = expired
        
    def load(self): 
        with open(self.invoice_name, 'rb') as i:
            self.data = pickle.load(i)

        with open(self.expired_name, 'r') as e:
            self.data_expired =  e.read().splitlines()
        self.data_expired = [int(value) for value in self.data_expired[0].split(', ')]
        
    def transform_and_return(self):
        transformed_data = []
        for invoice in self.data: 
            # https://chatgpt.com/share/b5fd090b-2298-442e-8252-808aa6672a51
            is_expired = 1 if invoice['id'] in self.data_expired else 0
            if 'items' in invoice and invoice['items']: 
                invoice_total = sum((self.convert_to_int(item['item']['unit_price']) if item['item']['unit_price'] != 'ten' else 10) * (self.convert_to_int(item['quantity']) if item['quantity'] != 'ten' else 10) for item in invoice['items'])
            else: 
                invoice_total = 0 
            if 'items' in invoice:
                for item in invoice['items']: 
                    invoiceitem_id = item['item']['id']
                    type_item = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}.get(item['item']['type'], 'Unknown')
                    invoiceitem_name = item['item']['name']
                    unit_price = self.convert_to_int(item['item']['unit_price']) if item['item']['unit_price'] != 'ten' else 10
                    quantity = self.convert_to_int(item['quantity']) if item['quantity'] != 'ten' else 10
                    total_price = unit_price * quantity
                    if invoice_total > 0:
                        percentage_in_invoice = total_price / invoice_total 
                    else: 
                        percentage_in_invoice = 0
                    # https://chatgpt.com/share/d8f7beff-1ea0-48cb-8e3e-5088f44853c7
                    transformed_data.append([invoice['id'], 
                                             pd.to_datetime(invoice['created_on'], errors='coerce'),
                                             invoiceitem_id,
                                             invoiceitem_name, 
                                             type_item,
                                             unit_price,
                                             total_price,
                                             percentage_in_invoice,
                                             is_expired])
        column_names = ['invoice_id', 'created_on', 'invoiceitem_id', 'invoiceitem_name', 'type', 'unit_price', 'total_price', 'percentage_in_invoice', 'is_expired'] 
        self.df = pd.DataFrame(transformed_data, columns=column_names) 
        self.df.sort_values(by=['invoice_id', 'invoiceitem_id'], inplace=True)
        return self.df

    def convert_to_int(self, value):
        if value == 'ten':
            return 10
        elif value == 'five':
            return 5
        try:
            return int(value)
        except ValueError:
            return 0

       

In [6]:
extractor = DataExtractor("invoices_new.pkl", "expired_invoices.txt")
extractor.load()
data_frame = extractor.transform_and_return()
print(data_frame.head()) 

    invoice_id created_on  invoiceitem_id invoiceitem_name       type  \
292     301695 2019-04-26          103215        ii_103215      Other   
290     301695 2019-04-26          166227        ii_166227  Equipment   
293     301695 2019-04-26          171394        ii_171394      Other   
291     301695 2019-04-26          195625        ii_195625      Other   
130     304245 2019-03-17          121446        ii_121446    Service   

     unit_price  total_price  percentage_in_invoice  is_expired  
292         135          945               0.374851           0  
290         118          708               0.280841           0  
293         128          128               0.050774           0  
291         148          740               0.293534           0  
130         158          632               0.128429           0  


In [7]:
# https://chatgpt.com/share/b59b8660-f880-4a49-b767-2a6a988d99f4
for invoice in invoices:
    if 'items' in invoice:
        for item in invoice['items']:
            unit_price_type = type(item['item']['unit_price'])
            quantity_type = type(item['quantity'])
            # print("  Item:", item['item']['name'])
            # print("    unit_price type:", unit_price_type)
            # print("    quantity type:", quantity_type)
    else:
        print("No items found for this invoice")

No items found for this invoice
No items found for this invoice


In [9]:
# https://chatgpt.com/share/f842e7fb-3078-4b3f-b0f5-5aebff6ee53d

for invoice in invoices:
    # print(f"Invoice ID: {invoice.get('id', 'N/A')}")
    items = invoice.get('items', [])  # Check if 'items' key exists, if not, default to an empty list
    for item in items:
        quantity = item.get('quantity', 'N/A')  # Get the quantity, default to 'N/A' if not found
        unit_price = item.get('item', {}).get('unit_price', 'N/A')  # Get the unit price, default to 'N/A' if not found
        if not isinstance(quantity, (int, float)):
            print(f"Non-numeric quantity found: {quantity}")
        if not isinstance(unit_price, (int, float)):
            print(f"Non-numeric unit price found: {unit_price}")
    # print()


Non-numeric quantity found: ten
Non-numeric quantity found: five
Non-numeric quantity found: ten
Non-numeric quantity found: ten
